In [2]:
from sentence_transformers import SentenceTransformer, util
from transformers import BertTokenizer
import pandas as pd
import numpy as np
import nltk
import math
from joblib import Parallel, delayed

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')

from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer

import torch
import time


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [3]:
df_assunto= pd.read_table("dados-conle-anonimizado-assunto-notnull - dados-conle-anonimizado-assunto-notnull.tsv",sep=";")

In [4]:
base = pd.read_csv("base_20230428_douglas.csv", sep=',')

In [5]:
base

,codProposicao,txtSiglaTipo,numAno,numNumero,txtNome,txtEmenta,txtExplicacaoEmenta,txtIndexacao,txtInteiroTeor
0,16357,PL,1999,1165,PL 1165/1999,"Altera dispositivo da Lei nº 8.987, de 13 de f...",Estabelece que as concessionárias disponibiliz...,"Alteração, Lei das Concessões de Serviços Públ...",Ofício nº 1416 (SF) ...
1,19098,PL,1992,3097,PL 3097/1992,Dispõe sobre a eleição de diretores de fundos ...,NaN,"NORMAS, ELEIÇÃO DIRETA, EMPREGADO, APOSENTADO,...",COMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E DE REDAÇÃ...
2,20464,PL,2000,3927,PL 3927/2000,Altera a composição dos Tribunais Regionais do...,"Altera a composição do TRT da 5ª região, 6ª re...","Alteração, Lei Federal, composição, Tribunal R...","COMISSÃO DE TRABALHO, DE ADMINISTRAÇÃO E SERVI..."
3,20683,PL,1998,4117,PL 4117/1998,Dispõe sobre o acesso a ambientes de uso colet...,NaN,"Autorização, pessoa portadora de deficiência, ...",Câmara dos Deputados\n Departamento ...
4,20857,PL,1998,4395,PL 4395/1998,Estabelece as Diretrizes Nacionais de Defesa C...,NaN,"Fixação, diretrizes, defesa civil, (Sindec), d...",- 1 - \nCOMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E D...
...,...,...,...,...,...,...,...,...,...
57104,2358873,PL,2023,2233,PL 2233/2023,Altera o parágrafo único do art. 71 do Código ...,NaN,NaN,1 \n \n \n \n \nCÂMARA DOS DEPUTADOS \nPROJET...
57105,2358874,PL,2019,3616,PL 3616/2019,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,NaN,"Altera a Lei nº 9.503, de 23 de setembro\nde 1..."
57106,2358875,PL,2019,1822,PL 1822/2019,"Altera a Lei nº 11.340, de 7 de agosto de 2006...",NaN,NaN,"Altera a Lei nº 11.340, de 7 de agosto de\n200..."
57107,2358877,PL,2019,3815,PL 3815/2019,"Altera a Lei nº 7.565, de 19 de dezembro de 19...",NaN,NaN,"Altera a Lei nº 7.565, de 19 de dezembro\nde ..."


In [6]:
model = SentenceTransformer("sentence-transformers/nli-bert-base")

In [7]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [8]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

In [9]:
device_gpu = torch.device("cuda")

## Search

In [10]:
def search(query, data_emb,base,top_k=20):
    #hits = util.semantic_search(query_emb,data,top_k=k)
    query_emb = model.encode(query,convert_to_tensor=True,device=device_gpu)
    hits = util.cos_sim(query_emb,data_emb)[0]
    rank = torch.topk(hits, k=top_k)
    
    
    top_n = list()
    for i in range(top_k):
        dado = dict()
        dado['txtNome'] = base.iloc[rank.indices[i].item(),4]
        dado['score'] = rank.values[i].item()
        top_n.append(dado)
    return top_n
    

## Recall

In [11]:
def verificar(y,top_n):
    for d in top_n:
        if y == d["txtNome"]:
            return 1
    return 0

In [12]:
def avaliacaoRecall(isPre,top_k=20):

    quant_encontrado=0
    quant_relevante =0
    a = time.time()
    for l,x in zip(y,X):
    
        query_tokenized = x
        if isPre:
            query_tokenized = preprocess(x)
            
        top_n = search(query_tokenized,data_emb,base,top_k)
                
        quant_relevante+=1
        quant_encontrado+=verificar(l,top_n)
    d = time.time()
    print("Duração: %f" %(d-a))
    recall = quant_encontrado / quant_relevante
    print("Recall: "+str(recall))

In [13]:
def avaliacaoRR(isPre):

    quant_encontrado=0
    for l,x in zip(y,X):
    
        query_tokenized = x
        if isPre:
            query_tokenized = preprocess(x)
            
        top_n = search(query_tokenized,data_emb,base)
        if top_n[0]["txtNome"].strip() == l.strip():
            quant_encontrado+=1
    rr = quant_encontrado / 295
    print("RR: %f" % (rr))

In [14]:
def avaliacaoPrecision(isPre,top_k=20):

    soma=0
    for l,x in zip(y,X):
    
        query_tokenized = x
        if isPre:
            query_tokenized = preprocess(x)
            
        top_n = search(query_tokenized,data_emb,base,top_k)
        encontrado=verificar(l,top_n)
        soma+=(encontrado/top_k)
    precision = soma / 295
    print("Precision: "+str(precision))

In [15]:
def avaliacaoMAP(isPre,top_k=20):
    l_v = list()
    labels_nsL = list()
    
    for l,x in zip(y,X):
    
        query_tokenized = x
        if isPre:
            query_tokenized = preprocess(x)
            
        top_n = search(query_tokenized,data_emb,base,top_k)
        l_v.append(l)
        labels_nsL.append([doc["txtNome"] for doc in top_n])
        
    denominador = 0
    encontrou=False
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        quant=0
        for k in x:
            if str(k).strip() == Y.strip():
                quant=1
                encontrou=True
            denominador+=1
            calc=quant/denominador
            if encontrou:
                calc = calc * 1
            else:
                calc = calc * 0
            soma+=calc
            encontrou = False
    MAP = soma /295
    print("MAP@ %d: %f" % (top_k,MAP))

In [163]:
data = base.txtInteiroTeor.fillna("Em branco")        # Trocar com nome da coluna desejada

## Config 1

In [164]:
a = time.time()

In [165]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [166]:
d = time.time()

In [167]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 2156.972861


## Avaliação

### Recall

In [168]:
avaliacaoRecall(False,top_k=5)

Duração: 21.865000
Recall: 0.020338983050847456


In [169]:
avaliacaoRecall(False,top_k=10)

Duração: 22.396000
Recall: 0.02711864406779661


In [170]:
avaliacaoRecall(False,top_k=20)

Duração: 22.964996
Recall: 0.03389830508474576


### MAP

In [171]:
avaliacaoMAP(False,top_k=5)

MAP@ 5: 0.000095


In [172]:
avaliacaoMAP(False,top_k=10)

MAP@ 10: 0.000054


In [173]:
avaliacaoMAP(False,top_k=20)

MAP@ 20: 0.000029


### Precision

In [174]:
avaliacaoPrecision(False,top_k=5)

Precision: 0.004067796610169492


In [175]:
avaliacaoPrecision(False,top_k=10)

Precision: 0.0027118644067796608


In [176]:
avaliacaoPrecision(False,top_k=20)

Precision: 0.0016949152542372879


### RR

In [177]:
avaliacaoRR(False)

RR: 0.010169


## Config 5

In [178]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return " ".join(terms)

In [179]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [180]:
a = time.time()

In [181]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [182]:
d = time.time()

In [183]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 2144.586112


## Avaliação

### Recall

In [184]:
avaliacaoRecall(True,top_k=5)

Duração: 20.787000
Recall: 0.01694915254237288


In [185]:
avaliacaoRecall(True,top_k=10)

Duração: 21.974998
Recall: 0.020338983050847456


In [186]:
avaliacaoRecall(True,top_k=20)

Duração: 22.474994
Recall: 0.03389830508474576


### MAP

In [187]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000022


In [188]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000022


In [189]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000033


### Precision

In [190]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.003389830508474576


In [191]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.002033898305084746


In [192]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.0016949152542372879


### RR

In [193]:
avaliacaoRR(True)

RR: 0.003390


## Config 0

In [194]:
def preprocess(txt):
    txt = str(txt)
    
    stopwords = nltk.corpus.stopwords.words("portuguese")
   
    terms = word_tokenize(txt)
    terms = [word for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [195]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [196]:
a = time.time()

In [197]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [198]:
d = time.time()

In [199]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 2187.252002


## Avaliação

### Recall

In [200]:
avaliacaoRecall(True,top_k=5)

Duração: 23.678994
Recall: 0.023728813559322035


In [201]:
avaliacaoRecall(True,top_k=10)

Duração: 22.563998
Recall: 0.03389830508474576


In [202]:
avaliacaoRecall(True,top_k=20)

Duração: 23.186998
Recall: 0.04067796610169491


### MAP

In [203]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000101


In [204]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000060


In [205]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000034


### Precision

In [206]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.0047457627118644066


In [207]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.0033898305084745757


In [208]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.002033898305084746


### RR

In [209]:
avaliacaoRR(True)

RR: 0.006780


# TxtEmenta

In [16]:
data = base.txtEmenta.fillna("Em branco")

## Config 1

In [17]:
a = time.time()

In [18]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [19]:
d = time.time()

In [20]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 506.237767


## Avaliação

### Recall

In [21]:
avaliacaoRecall(False,top_k=5)

Duração: 9.413576
Recall: 0.09152542372881356


In [22]:
avaliacaoRecall(False,top_k=10)

Duração: 9.636001
Recall: 0.09830508474576272


In [23]:
avaliacaoRecall(False,top_k=20)

Duração: 11.634689
Recall: 0.11186440677966102


### MAP

In [24]:
avaliacaoMAP(False,top_k=5)

MAP@ 5: 0.001100


In [25]:
avaliacaoMAP(False,top_k=10)

MAP@ 10: 0.000586


In [26]:
avaliacaoMAP(False,top_k=20)

MAP@ 20: 0.000311


### Precision

In [27]:
avaliacaoPrecision(False,top_k=5)

Precision: 0.018305084745762718


In [28]:
avaliacaoPrecision(False,top_k=10)

Precision: 0.009830508474576276


In [29]:
avaliacaoPrecision(False,top_k=20)

Precision: 0.005593220338983054


### RR

In [30]:
avaliacaoRR(False)

RR: 0.050847


In [31]:
avaliacaoRR(False)

RR: 0.050847


## Config 5

In [32]:
data = base.txtEmenta.fillna("Em branco")

In [33]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return " ".join(terms)

In [34]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [35]:
a = time.time()

In [36]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [37]:
d = time.time()

In [38]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 507.365143


## Avaliação

### Recall

In [39]:
avaliacaoRecall(True,top_k=5)

Duração: 11.432000
Recall: 0.08135593220338982


In [40]:
avaliacaoRecall(True,top_k=10)

Duração: 11.752998
Recall: 0.10508474576271186


In [41]:
avaliacaoRecall(True,top_k=20)

Duração: 11.372882
Recall: 0.12542372881355932


### MAP

In [42]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000939


In [43]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000553


In [44]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000303


### Precision

In [45]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.01627118644067797


In [46]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.01050847457627119


In [47]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.00627118644067797


### RR

In [48]:
avaliacaoRR(True)

RR: 0.030508


## Config 0

In [49]:
data = base.txtEmenta.fillna("Em branco")

In [50]:
def preprocess(txt):
    txt = str(txt)
    
    stopwords = nltk.corpus.stopwords.words("portuguese")
   
    terms = word_tokenize(txt)
    terms = [word for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [51]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [52]:
a = time.time()

In [53]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [54]:
d = time.time()

In [55]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 606.064136


## Avaliação

### Recall

In [56]:
avaliacaoRecall(True,top_k=5)

Duração: 78.287998
Recall: 0.09152542372881356


In [57]:
avaliacaoRecall(True,top_k=10)

Duração: 98.992996
Recall: 0.09491525423728814


In [58]:
avaliacaoRecall(True,top_k=20)

Duração: 119.728519
Recall: 0.1152542372881356


### MAP

In [59]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000993


In [60]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000548


In [61]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000297


### Precision

In [62]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.018305084745762718


In [63]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.009491525423728818


In [64]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.0057627118644067825


### RR

In [65]:
avaliacaoRR(True)

RR: 0.047458


# TxtIndexacao

In [66]:
data = base.txtIndexacao.fillna("Em branco")

## Config 1

In [67]:
a = time.time()

In [68]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [69]:
d = time.time()

In [70]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 2052.435001


## Avaliação

### Recall

In [71]:
avaliacaoRecall(False,top_k=5)

Duração: 71.077000
Recall: 0.04067796610169491


In [72]:
avaliacaoRecall(False,top_k=10)

Duração: 59.080999
Recall: 0.05423728813559322


In [73]:
avaliacaoRecall(False,top_k=20)

Duração: 52.959999
Recall: 0.07457627118644068


### MAP

In [74]:
avaliacaoMAP(False,top_k=5)

MAP@ 5: 0.000069


In [75]:
avaliacaoMAP(False,top_k=10)

MAP@ 10: 0.000046


In [76]:
avaliacaoMAP(False,top_k=20)

MAP@ 20: 0.000046


### Precision

In [77]:
avaliacaoPrecision(False,top_k=5)

Precision: 0.008135593220338983


In [78]:
avaliacaoPrecision(False,top_k=10)

Precision: 0.005423728813559323


In [79]:
avaliacaoPrecision(False,top_k=20)

Precision: 0.003728813559322035


### RR

In [80]:
avaliacaoRR(False)

RR: 0.020339


## Config 5

In [81]:
data = base.txtIndexacao.fillna("Em branco")

In [82]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return " ".join(terms)

In [83]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [84]:
a = time.time()

In [85]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [86]:
d = time.time()

In [87]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 1840.177679


## Avaliação

### Recall

In [88]:
avaliacaoRecall(True,top_k=5)

Duração: 21.955994
Recall: 0.04406779661016949


In [89]:
avaliacaoRecall(True,top_k=10)

Duração: 20.845002
Recall: 0.05084745762711865


In [90]:
avaliacaoRecall(True,top_k=20)

Duração: 24.479996
Recall: 0.07796610169491526


### MAP

In [91]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000077


In [92]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000043


In [93]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000039


### Precision

In [94]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.008813559322033898


In [95]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.005084745762711865


In [96]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.0038983050847457637


### RR

In [97]:
avaliacaoRR(True)

RR: 0.027119


## Config 0

In [98]:
data = base.txtIndexacao.fillna("Em branco")

In [99]:
def preprocess(txt):
    txt = str(txt)
    
    stopwords = nltk.corpus.stopwords.words("portuguese")
   
    terms = word_tokenize(txt)
    terms = [word for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [100]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [101]:
a = time.time()

In [102]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [103]:
d = time.time()

In [104]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 1266.961002


## Avaliação

### Recall

In [105]:
avaliacaoRecall(True,top_k=5)

Duração: 22.195046
Recall: 0.04745762711864407


In [106]:
avaliacaoRecall(True,top_k=10)

Duração: 23.435001
Recall: 0.06440677966101695


In [107]:
avaliacaoRecall(True,top_k=20)

Duração: 22.897965
Recall: 0.07796610169491526


### MAP

In [108]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.000080


In [109]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000052


In [110]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000040


### Precision

In [111]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.009491525423728815


In [112]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.006440677966101697


In [113]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.0038983050847457637


### RR

In [114]:
avaliacaoRR(True)

RR: 0.023729


# Emenda + Indexacao

In [115]:
base['txtIndexacao'] = base['txtEmenta'] + base['txtIndexacao']

In [116]:
data = base.txtIndexacao.fillna("Em branco")

## Config 1

In [117]:
a = time.time()

In [118]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [119]:
d = time.time()

In [120]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 1872.611599


## Avaliação

### Recall

In [121]:
avaliacaoRecall(False,top_k=5)

Duração: 22.220999
Recall: 0.0711864406779661


In [122]:
avaliacaoRecall(False,top_k=10)

Duração: 21.160986
Recall: 0.07796610169491526


In [123]:
avaliacaoRecall(False,top_k=20)

Duração: 24.855001
Recall: 0.09152542372881356


### MAP

In [124]:
avaliacaoMAP(False,top_k=5)

MAP@ 5: 0.000992


In [125]:
avaliacaoMAP(False,top_k=10)

MAP@ 10: 0.000550


In [126]:
avaliacaoMAP(False,top_k=20)

MAP@ 20: 0.000295


### Precision

In [127]:
avaliacaoPrecision(False,top_k=5)

Precision: 0.014237288135593223


In [128]:
avaliacaoPrecision(False,top_k=10)

Precision: 0.007796610169491527


In [129]:
avaliacaoPrecision(False,top_k=20)

Precision: 0.0045762711864406795


### RR

In [130]:
avaliacaoRR(False)

RR: 0.027119


## Config 5

In [131]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return " ".join(terms)

In [132]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [133]:
a = time.time()

In [134]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [135]:
d = time.time()

In [136]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 1890.772318


## Avaliação

### Recall

In [137]:
avaliacaoRecall(True,top_k=5)

Duração: 21.685998
Recall: 0.06779661016949153


In [138]:
avaliacaoRecall(True,top_k=10)

Duração: 20.893996
Recall: 0.09152542372881356


In [139]:
avaliacaoRecall(True,top_k=20)

Duração: 22.497998
Recall: 0.10847457627118644


### MAP

In [140]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.001025


In [141]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000563


In [142]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000305


### Precision

In [143]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.013559322033898308


In [144]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.009152542372881359


In [145]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.005423728813559325


### RR

In [146]:
avaliacaoRR(True)

RR: 0.020339


## Config 0

In [147]:
def preprocess(txt):
    txt = str(txt)
    
    stopwords = nltk.corpus.stopwords.words("portuguese")
   
    terms = word_tokenize(txt)
    terms = [word for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [148]:
data_preprocess = Parallel(n_jobs=4)(delayed(preprocess)(doc) for doc in data)

In [149]:
a = time.time()

In [150]:
data_emb = model.encode(data,convert_to_tensor=True,device=device_gpu)

In [151]:
d = time.time()

In [152]:
print("Duração da indexação: %f" % (d-a))

Duração da indexação: 1878.885997


## Avaliação

### Recall

In [153]:
avaliacaoRecall(True,top_k=5)

Duração: 22.370999
Recall: 0.06779661016949153


In [154]:
avaliacaoRecall(True,top_k=10)

Duração: 23.418993
Recall: 0.08813559322033898


In [155]:
avaliacaoRecall(True,top_k=20)

Duração: 24.793999
Recall: 0.1016949152542373


### MAP

In [156]:
avaliacaoMAP(True,top_k=5)

MAP@ 5: 0.001002


In [157]:
avaliacaoMAP(True,top_k=10)

MAP@ 10: 0.000556


In [158]:
avaliacaoMAP(True,top_k=20)

MAP@ 20: 0.000296


### Precision

In [159]:
avaliacaoPrecision(True,top_k=5)

Precision: 0.013559322033898308


In [160]:
avaliacaoPrecision(True,top_k=10)

Precision: 0.008813559322033902


In [161]:
avaliacaoPrecision(True,top_k=20)

Precision: 0.005084745762711867


### RR

In [162]:
avaliacaoRR(True)

RR: 0.023729
